# The capital asset pricing model and the security market line

In [1]:
""" 
Linear regression with SciPy 
"""
from scipy import stats

stock_returns = [0.065, 0.0265, -0.0593, -0.001, 0.0346]
mkt_returns = [0.055, -0.09, -0.041, 0.045, 0.022]
beta, alpha, r_value, p_value, std_err = \
    stats.linregress(stock_returns, mkt_returns)

In [2]:
print(beta, alpha)

0.5077431878770808 -0.008481900352462384


# Multivariate linear regression of factor models

In [7]:
""" 
Least squares regression with statsmodels 
"""
import numpy as np
import statsmodels.api as sm

# Generate some sample data
num_periods = 9
all_values = np.array([np.random.random(8) \
                       for i in range(num_periods)])

# Filter the data
y_values = all_values[:, 0] # First column values as Y
x_values = all_values[:, 1:] # All other values as X
x_values = sm.add_constant(x_values) # Include the intercept
print(x_values.shape)
x_values

(9, 8)


array([[1.        , 0.11213189, 0.35456165, 0.85688116, 0.00457458,
        0.20271716, 0.23332719, 0.62053555],
       [1.        , 0.42119736, 0.46164868, 0.15633123, 0.65539847,
        0.9753709 , 0.52887925, 0.72307649],
       [1.        , 0.0206164 , 0.29997799, 0.58508238, 0.18062494,
        0.3125823 , 0.68612099, 0.86464407],
       [1.        , 0.19757505, 0.28904671, 0.72671822, 0.21022532,
        0.84083565, 0.59285091, 0.13967666],
       [1.        , 0.6806411 , 0.84313875, 0.48043954, 0.5097927 ,
        0.18971553, 0.61201954, 0.6430078 ],
       [1.        , 0.03830537, 0.6838201 , 0.63408164, 0.23086406,
        0.28827615, 0.64074769, 0.66420321],
       [1.        , 0.08813594, 0.46550674, 0.62310364, 0.10038213,
        0.23839102, 0.7916267 , 0.8315974 ],
       [1.        , 0.03139812, 0.87854304, 0.30036601, 0.77519675,
        0.69548321, 0.013516  , 0.77393578],
       [1.        , 0.12427252, 0.15841013, 0.10894916, 0.53139299,
        0.23875292, 0.738700

In [8]:
print(y_values.shape)
y_values

(9,)


array([0.56584596, 0.39749769, 0.71733757, 0.06098289, 0.77129806,
       0.18403918, 0.84194011, 0.0070909 , 0.44103028])

In [9]:
results = sm.OLS(y_values, x_values).fit() # Regress and fit the model
print(results.summary())

                            OLS Regression Results                            
Dep. Variable:                      y   R-squared:                       0.940
Model:                            OLS   Adj. R-squared:                  0.517
Method:                 Least Squares   F-statistic:                     2.223
Date:                Tue, 21 Jul 2020   Prob (F-statistic):              0.476
Time:                        03:56:08   Log-Likelihood:                 10.969
No. Observations:                   9   AIC:                            -5.938
Df Residuals:                       1   BIC:                            -4.360
Df Model:                           7                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.1305      1.701      0.077      0.9

/usr/local/lib/python3.6/dist-packages/scipy/stats/stats.py:1535: UserWarning: kurtosistest only valid for n>=20 ... continuing anyway, n=9
  "anyway, n=%i" % int(n))


In [10]:
print(results.params)

[ 0.13052656  0.80156336 -0.85511034  0.25539379  0.27662829 -0.57030795
  0.17641093  0.84542403]


# Linear optimization

## A maximization example with linear programming

In [12]:
!pip install pulp

     |████████████████████████████████| 40.6MB 97kB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
  Created wheel for amply: filename=amply-0.1.2-cp36-none-any.whl size=16572 sha256=a7de7fd30899031463da30854fb698f53648e943dc956d7b7d8afb4c2e974c77
  Stored in directory: /root/.cache/pip/wheels/84/18/f7/e5c3ed13ed5bb721763f77d4a924331d59ef115ce61c9d26eb
Successfully built amply


In [17]:
""" 
A linear optimization problem with 2 variables
"""
import pulp

x = pulp.LpVariable('x', lowBound=0)
y = pulp.LpVariable('y', lowBound=0)
x

x

In [18]:
problem = pulp.LpProblem(
    'A simple maximization objective', 
    pulp.LpMaximize)
problem += 3*x + 2*y, 'The objective function'
problem += 2*x + y <= 100, '1st constraint'
problem += x + y <= 80, '2nd constraint'
problem += x <= 40, '3rd constraint'
problem

/usr/local/lib/python3.6/dist-packages/pulp/pulp.py:1190: UserWarning: Spaces are not permitted in the name. Converted to '_'
  warnings.warn("Spaces are not permitted in the name. Converted to '_'")


A_simple_maximization_objective:
MAXIMIZE
3*x + 2*y + 0
SUBJECT TO
1st_constraint: 2 x + y <= 100

2nd_constraint: x + y <= 80

3rd_constraint: x <= 40

VARIABLES
x Continuous
y Continuous

In [19]:
problem.solve()
print("Maximization Results:")
for variable in problem.variables():
    print(variable.name, '=', variable.varValue)

Maximization Results:
x = 20.0
y = 60.0


## A minimization example with integer programming

In [26]:
""" 
An example of implementing an integer 
programming model with binary conditions 
"""
import pulp

dealers = ['X', 'Y', 'Z']
variable_costs = {'X': 500, 'Y': 350, 'Z': 450}
fixed_costs = {'X': 4000, 'Y': 2000, 'Z': 6000}

# Define PuLP variables to solve
quantities = pulp.LpVariable.dicts('quantity', 
                                   dealers, 
                                   lowBound=0,
                                   cat=pulp.LpInteger)
is_orders = pulp.LpVariable.dicts('orders', 
                                  dealers,
                                  cat=pulp.LpBinary)

In [27]:
quantities

{'X': quantity_X, 'Y': quantity_Y, 'Z': quantity_Z}

In [28]:
is_orders

{'X': orders_X, 'Y': orders_Y, 'Z': orders_Z}

In [21]:
"""
This is an example of implementing an integer programming model
with binary variables the wrong way.
"""
# Initialize the model with constraints
model = pulp.LpProblem('A cost minimization problem',
                       pulp.LpMinimize)
model += sum([(variable_costs[i] * \
               quantities[i] + \
               fixed_costs[i])*is_orders[i] \
              for i in dealers]), 'Minimize portfolio cost'
model += sum([quantities[i] for i in dealers]) == 150\
    , 'Total contracts required'
model += 30 <= quantities['X'] <= 100\
    , 'Boundary of total volume of X'
model += 30 <= quantities['Y'] <= 90\
    , 'Boundary of total volume of Y'
model += 30 <= quantities['Z'] <= 70\
    , 'Boundary of total volume of Z'
model.solve()  # You will get an error running this code!

/usr/local/lib/python3.6/dist-packages/pulp/pulp.py:1190: UserWarning: Spaces are not permitted in the name. Converted to '_'
  warnings.warn("Spaces are not permitted in the name. Converted to '_'")


TypeError: ignored

In [22]:
"""
This is an example of implementing an 
IP model with binary variables the correct way.
"""
# Initialize the model with constraints
model = pulp.LpProblem('A cost minimization problem',
                       pulp.LpMinimize)
model += sum(
    [variable_costs[i]*quantities[i] + \
         fixed_costs[i]*is_orders[i] for i in dealers])\
    , 'Minimize portfolio cost'
model += sum([quantities[i] for i in dealers]) == 150\
    ,  'Total contracts required'
model += is_orders['X']*30 <= quantities['X'] <= \
    is_orders['X']*100, 'Boundary of total volume of X'
model += is_orders['Y']*30 <= quantities['Y'] <= \
    is_orders['Y']*90, 'Boundary of total volume of Y'
model += is_orders['Z']*30 <= quantities['Z'] <= \
    is_orders['Z']*70, 'Boundary of total volume of Z'
model.solve()

/usr/local/lib/python3.6/dist-packages/pulp/pulp.py:1190: UserWarning: Spaces are not permitted in the name. Converted to '_'
  warnings.warn("Spaces are not permitted in the name. Converted to '_'")


1

In [23]:
print('Minimization Results:')
for variable in model.variables():
    print(variable, '=', variable.varValue)

print('Total cost:',  pulp.value(model.objective))

Minimization Results:
orders_X = 0.0
orders_Y = 1.0
orders_Z = 1.0
quantity_X = 0.0
quantity_Y = 90.0
quantity_Z = 60.0
Total cost: 66500.0


# Solving linear equations using matrices

In [31]:
""" 
Linear algebra with NumPy matrices 
"""
import numpy as np

A = np.array([[2, 1, 1],[1, 3, 2],[1, 0, 0]])
B = np.array([4, 5, 6])
print(A.shape)
print(B.shape)
print(A)
print(B)

(3, 3)
(3,)
[[2 1 1]
 [1 3 2]
 [1 0 0]]
[4 5 6]


Use the `linalg.solve` function of NumPy to solve a system of linear scalar
equations:

In [30]:
print(np.linalg.solve(A, B))

[  6.  15. -23.]


# The LU decomposition

In [32]:
 """ 
LU decomposition with SciPy 
"""
import numpy as np
import scipy.linalg as linalg


# Define A and B
A = np.array([
    [2., 1., 1.],
    [1., 3., 2.],
    [1., 0., 0.]])
B = np.array([4., 5., 6.])

# Perform LU decompositiopn
LU = linalg.lu_factor(A)
x = linalg.lu_solve(LU, B)

In [33]:
print(x)

[  6.  15. -23.]


In [34]:
import scipy

P, L, U = scipy.linalg.lu(A)

print('P=\n', P)
print('L=\n', L)
print('U=\n', U)

P=
 [[1. 0. 0.]
 [0. 1. 0.]
 [0. 0. 1.]]
L=
 [[ 1.   0.   0. ]
 [ 0.5  1.   0. ]
 [ 0.5 -0.2  1. ]]
U=
 [[ 2.   1.   1. ]
 [ 0.   2.5  1.5]
 [ 0.   0.  -0.2]]


# The Cholesky decomposition

In [35]:
""" 
Cholesky decomposition with NumPy 
"""
import numpy as np

A = np.array([
    [10., -1., 2., 0.],
    [-1., 11., -1., 3.],
    [2., -1., 10., -1.],
    [0., 3., -1., 8.]])
B = np.array([6., 25., -11., 15.])

L = np.linalg.cholesky(A)

In [36]:
print(L)

[[ 3.16227766  0.          0.          0.        ]
 [-0.31622777  3.3015148   0.          0.        ]
 [ 0.63245553 -0.24231301  3.08889696  0.        ]
 [ 0.          0.9086738  -0.25245792  2.6665665 ]]


In [37]:
print(np.dot(L, L.T.conj()))  # A=L.L*

[[10. -1.  2.  0.]
 [-1. 11. -1.  3.]
 [ 2. -1. 10. -1.]
 [ 0.  3. -1.  8.]]


In [38]:
y = np.linalg.solve(L, B)  # L.L*.x=B; When L*.x=y, then L.y=B

In [39]:
x = np.linalg.solve(L.T.conj(), y)  # x=L*'.y

In [40]:
print(x)

[ 1.  2. -1.  1.]


In [41]:
print(np.mat(A) * np.mat(x).T)  # B=Ax

[[  6.]
 [ 25.]
 [-11.]
 [ 15.]]


# The QR decomposition

In [42]:
""" 
QR decomposition with scipy 
"""
import numpy as np
import scipy.linalg as linalg


A = np.array([
    [2., 1., 1.],
    [1., 3., 2.],
    [1., 0., 0]])
B = np.array([4., 5., 6.])

Q, R = scipy.linalg.qr(A)  # QR decomposition
y = np.dot(Q.T, B)  # Let y=Q'.B
x = scipy.linalg.solve(R, y)  # Solve Rx=y

In [43]:
print(x)

[  6.  15. -23.]


# Solving with other matrix algebra methods

## The Jacobi method

In [44]:
"""
Solve Ax=B with the Jacobi method 
"""
import numpy as np

def jacobi(A, B, n, tol=1e-10):
    # Initializes x with zeroes with same shape and type as B
    x = np.zeros_like(B)

    for iter_count in range(n):
        x_new = np.zeros_like(x)
        for i in range(A.shape[0]):
            s1 = np.dot(A[i, :i], x[:i])
            s2 = np.dot(A[i, i + 1:], x[i + 1:])
            x_new[i] = (B[i] - s1 - s2) / A[i, i]

        if np.allclose(x, x_new, tol):
            break

        x = x_new

    return x

In [45]:
A = np.array([
    [10., -1., 2., 0.], 
    [-1., 11., -1., 3.], 
    [2., -1., 10., -1.], 
    [0.0, 3., -1., 8.]])
B = np.array([6., 25., -11., 15.])
n = 25

In [46]:
x = jacobi(A, B, n)
print('x', '=', x)

x = [ 1.  2. -1.  1.]


# The Gauss-Seidel method

In [47]:
""" 
Solve Ax=B with the Gauss-Seidel method 
"""
import numpy as np


def gauss(A, B, n, tol=1e-10):
    L = np.tril(A)  # returns the lower triangular matrix of A
    U = A-L  # decompose A = L + U
    L_inv = np.linalg.inv(L)
    x = np.zeros_like(B)

    for i in range(n):
        Ux = np.dot(U, x)
        x_new = np.dot(L_inv, B - Ux)

        if np.allclose(x, x_new, tol):
            break

        x = x_new

    return x

In [48]:
A = np.array([
    [10., -1., 2., 0.], 
    [-1., 11., -1., 3.], 
    [2., -1., 10., -1.], 
    [0.0, 3., -1., 8.]])
B = np.array([6., 25., -11., 15.])
n = 100
x = gauss(A, B, n)

In [49]:
print('x', '=', x)

x = [ 1.  2. -1.  1.]
